In [8]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [9]:
product_type = "frozen"
num_time_periods = 7
test_article = '468a73f3'

In [10]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]
# article_data = article_data[(article_data['ARTICLE_ID'] == test_article)]

In [11]:
# constants definitions
if(product_type=="frozen"):
    warehouse_volume = 50
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 900
buffer_cost = 25
default_max_order = 10000

In [12]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [13]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [14]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [15]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [16]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start='2022-06-13', end='2022-06-18', freq='D')}).astype(str)
# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

# demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
# demand = dict((date_to_index[key],value) for (key,value) in demand.items())

demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

#### Greedy Approach

In [36]:
def GreedyApproach(item):
    item_demand = list(demand[item].values())
    fixed_cost = ordering_cost_fixed[item]
    variable_cost = ordering_cost_per_tu[item]
    cu_per_tu_item = cu_per_tu[item]
    disposal_cost = clearing_cost_per_cu[item]
    life = shelf_life[item]
    orders = {}
    order_date = 0
    obj_val = 0
    min_cost = float('inf')
    orders[order_date] = item_demand[order_date]
    #From 0 to 6 or from 0 to 365
    volume = 0
    print("warehouse_volume ",warehouse_volume)
    for t in range(order_date, len(item_demand)):
            if(volume < warehouse_volume):
                amount_order_tu=math.ceil(item_demand[t]/cu_per_tu_item);
                orders[t] = amount_order_tu
                cost_everyday=amount_order_tu*variable_cost+fixed_cost+(amount_order_tu*cu_per_tu_item-item_demand[t])*disposal_cost
                obj_val+= cost_everyday
                volume += amount_order_tu * cu_per_tu[item] * volume_per_cu[item]
                print("Print the demand according the periods that it is evaluating ",item_demand)
                print("Period t ",t)
                print("What is the demand on period t ",item_demand[t])
                print("How many CU are equal ",cu_per_tu_item)
                print("how much to ask ",amount_order_tu)
                print("Cost of ordering for TU ",amount_order_tu*variable_cost)
                print("Fixed Cost per order ",fixed_cost)
                print("Clearing Cost per CU",(amount_order_tu*cu_per_tu_item-item_demand[t])*disposal_cost)
            else:
                break
    return (obj_val)
obj_val = 0
for item in demand.keys():
    val = GreedyApproach(item)  
    obj_val += val
print(obj_val)

warehouse_volume  50
Weekly demand  [6.0, 14.0, 6.0, 0.0, 19.0, 4.0]
item_demand 6.0
relation  8
how much to ask  1
when to order  0
Cost of ordering for TU  1.1
Fixed Cost per order  9.0
Clearing Cost per CU 1.4
Weekly demand  [6.0, 14.0, 6.0, 0.0, 19.0, 4.0]
item_demand 14.0
relation  8
how much to ask  2
when to order  1
Cost of ordering for TU  2.2
Fixed Cost per order  9.0
Clearing Cost per CU 1.4
Weekly demand  [6.0, 14.0, 6.0, 0.0, 19.0, 4.0]
item_demand 6.0
relation  8
how much to ask  1
when to order  2
Cost of ordering for TU  1.1
Fixed Cost per order  9.0
Clearing Cost per CU 1.4
Weekly demand  [6.0, 14.0, 6.0, 0.0, 19.0, 4.0]
item_demand 0.0
relation  8
how much to ask  0
when to order  3
Cost of ordering for TU  0.0
Fixed Cost per order  9.0
Clearing Cost per CU 0.0
Weekly demand  [6.0, 14.0, 6.0, 0.0, 19.0, 4.0]
item_demand 19.0
relation  8
how much to ask  3
when to order  4
Cost of ordering for TU  3.3000000000000003
Fixed Cost per order  9.0
Clearing Cost per CU 3.5
We

#### Method 1

In [ ]:
class CostMatrixCell:
    def __init__(self) -> None:
        self.orders = {}
        self.order_date = None
        self.period_end = None
        self.cost = None
        self.avg_cost = None
        # self.extra_orders = []

class Order:
    def __init__(self) -> None:
        self.item = None
        self.quantity_tu = None
        self.date = None

# class ExtraOrder:
#     def __init__(self) -> None:
#         self.orders = {}
#         self.order_date = None
#         self.period_end = None


In [ ]:
n = 6
cost_matrix = [[CostMatrixCell() for __ in range(n)] for _ in range(n)]
for i in range(n):
    for j in range(n):
        if(i>j):
            cost_matrix[i][j] = None
        else:
            cost_matrix[i][j].order_date = i
            cost_matrix[i][j].period_end = j

def getOrders(cell):
    for t in range(cell.order_date, cell.period_end + 1):
        for item in demand[t].keys():
            if(item not in cell.orders.keys()):
                cell.orders[item] = 0
            cell.orders[item] += math.ceil(demand[t][item] / cu_per_tu[item])
            

def getCost(cell):
    cost = 0
    volume = 0
    for item in cell.orders.keys():
        life = shelf_life[item]
        cost += (ordering_cost_per_tu[item] * cell.orders[item]) + ordering_cost_fixed[item]
        volume += orders[item] * cu_per_tu[item] * volume_per_cu[item]
    buffer_cost = 0
    for t in range(cell.order_date,cell.period_end+1):
        if(volume > warehouse_volume):
            cost += (volume - warehouse_volume) * buffer_cost
            demand_volume = 0
            for item in demand[t].keys():
                demand_volume += volume_per_cu[item] * demand[t][item]
            volume -= demand_volume
        else:
            break
    
    cell.cost = cost
    cell.avg_cost = cost / (cell.period_end - cell.order_date + 1) 

for i in range(n):
    for j in range(n):
        if(type(cost_matrix[i][j])==CostMatrixCell):
            getCost(cost_matrix[i][j])

print(cost_matrix[0][0].cost)

# def getFirstMin(costs,first_index):
#     i = first_index
#     while(i < len(costs)-1 and costs[i].avg_cost > costs[i+1].avg_cost):
#         i = i+1
#     return (costs[i], i)

# obj_val = 0
# first_index = 0
# for i in range(n):
#     if(getFirstMin(cost_matrix[i], first_index)[1]+1 > n-1):
#         obj_val += getFirstMin(cost_matrix[i], first_index)[0].cost
#         break
#     obj_val += getFirstMin(cost_matrix[i], first_index)[0].cost
#     first_index = getFirstMin(cost_matrix[i], first_index)[1]+1

# obj_val

0


In [ ]:
n = 6
cost_matrix = [ [0] * n for _ in range(n)]
extra_orders_matrix = [ [None] * n for _ in range(n)]
for i in range(n):
    for j in range(n):
        if(i>j):
            cost_matrix[i][j] = float('inf')

# what and how much to order on day i to satisfy demand from day i to day j
def getOrders(i,j):
    orders = {}
    for t in range(i,j+1):
        for item in demand[t].keys():
            if(item not in orders.keys()):
                orders[item] = 0
            orders[item] += math.ceil(demand[t][item] / cu_per_tu[item])
    return orders

def getCost(i,j):
    orders = getOrders(i,j)
    cost = 0
    volume = 0 
    for item in orders.keys():
        life = shelf_life[item]
        cost += (ordering_cost_per_tu[item] * orders[item]) + ordering_cost_fixed[item]
        volume += orders[item] * cu_per_tu[item] * volume_per_cu[item]
    buffer_cost = 0
    for t in range(i,j+1):
        if(volume > warehouse_volume):
            cost += (volume - warehouse_volume) * buffer_cost
            demand_volume = 0
            for item in demand[t].keys():
                demand_volume += volume_per_cu[item] * demand[t][item]
            volume -= demand_volume
        else:
            break
    
    return cost, cost/(j-i+1) 

for i in range(n):
    for j in range(n):
        if(cost_matrix[i][j]!=float('inf')):
            cost_matrix[i][j] = getCost(i,j)

def getFirstMin(costs,first_index):
    i = first_index
    while(i < len(costs)-1 and costs[i][1] > costs[i+1][1]):
        i = i+1
    return (costs[i], i)

obj_val = 0
first_index = 0
for i in range(n):
    if(getFirstMin(cost_matrix[i], first_index)[1]+1 > n-1):
        obj_val += getFirstMin(cost_matrix[i], first_index)[0][0]
        break
    obj_val += getFirstMin(cost_matrix[i], first_index)[0][0]
    first_index = getFirstMin(cost_matrix[i], first_index)[1]+1

obj_val


10310.499999999996

In [ ]:
run_complete = False
obj_value = 0
order_date = 0

while(not run_complete):
    orders = {}
    min_cost = float('inf')
    for period_end in time_indexes[order_date:]:

        for item in demand[period_end].keys():
            if(item not in orders.keys()):
                orders[item] = 0
            orders[item] += math.ceil(demand[period_end][item] / cu_per_tu[item])

        cost = 0
        for item in orders:
            cost += (orders[item] * ordering_cost_per_tu[item]) + ordering_cost_fixed[item]
        avg_cost = cost / (period_end - order_date + 1)

        if(avg_cost <= min_cost):
            min_cost = avg_cost
            if(period_end==len(time_indexes)-1):
                obj_value += cost
                run_complete = True
        else:
            order_date = period_end
            break
obj_value

10310.499999999996

In [ ]:
run_complete = False
obj_value = 0
order_date = 0
while(not run_complete):
    ordered_items = set()
    min_cost = float('inf')
    for period_end in time_indexes[order_date:]:
        print(order_date, period_end)
        for item in demand[period_end].keys():
            if(demand[period_end][item] > 0):
                ordered_items.add(item)
        print(ordered_items)
        cost = 0
        for item in ordered_items:
            cost += ordering_cost_fixed[item]
        if(cost <= min_cost):
            min_cost = cost
            if(period_end==len(time_indexes)-1):
                run_complete = True
        else:
            order_date = period_end
            break
    obj_value += min_cost
obj_value

0 0
{'2cd380af', '50b96f4c', '499d4776', 'f5a9d9d9', '5280d290', 'a82fc331', 'b294630c', '189067f1', 'e270732d', '8e205b36', 'df254538', '1814bccd', '9364caed', '1c939a8e', '499ff4b3', 'd44ecf7a', '11545bca', 'e425a2dd', '781f2cb5', 'a98513e1', '982dc667', 'd0d666f6', '43a7945a', '7465367d', '327dfe80', '87bc979c', 'bb41e272', '359fe15b', '938f7d75', 'e0cfdaa5', '9718a0ea', '75a2af63', 'b43301d5', '875a176c', 'c28c3b5d', 'e71736a4', '10f2b453', '61567e8b', 'e394e81a', 'c936ca6b', '8339c92a', 'bedc5c28', '761fcc1e', '09e397f8', 'b13b307a', '020b50b0', '2410d366', 'c148f52c', '1377878b', 'c47954c7', 'b4387f22', '34ece327', '1d34848b', '44c369a4', 'd50b255e', '3038bf57', 'c9dafa61', '58499bdc', 'd4132dce', 'ba8d7c6d', '47afad28', '7f4d8743', 'a32ee9e0', 'fea8d9f0', '2d82f35e', 'd3522cdb', 'bd574fe0', '126334a5', 'c958c1bb', '68425ae4', '2f479fa5', 'e96aad87', '25b50b2a', '68311c41', '6547bcba', 'd396d0ba', 'a1136423', '097bbc43', '52c5bfab', 'b70207f6', '1e6e2355', '84293966', '1f9d7d64',

16537.4